In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [8]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import time
import pickle

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter

from sklearn import metrics
from sklearn.metrics import classification_report, f1_score

In [3]:
! nvidia-smi

Tue Nov 17 23:25:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.04    Driver Version: 455.23.04    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:05:00.0 Off |                    0 |
| N/A   23C    P8    26W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           On   | 00000000:06:00.0 Off |                    0 |
| N/A   

In [11]:
torch.manual_seed(42)
np.random.seed(42)

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Device:', device)
torch.backends.cudnn.benchmark = True

Device: cuda


In [16]:
from datasets import load_dataset

conll_dataset = load_dataset("conll2003")

Reusing dataset conll2003 (/opt/tmp/huggingface/datasets/conll2003/conll2003/1.0.0/26b70ce2b0f32cb35a27151dbfa2dbe88c82bcdaf8f29433bcdc612a9b314e83)


In [9]:
cn = pd.read_csv('conceptnet_en.csv')
cn_isa = pd.read_csv('data/conceptnet_isa.csv')

In [17]:
len(conll_dataset['train']), len(conll_dataset['validation']), len(conll_dataset['test'])

(14041, 3250, 3453)

In [18]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [22]:
nodes_embeddings = {'hope_gsvd': None,
                    'lap_eigmap_svd': None,
                    'lle_svd': None,
                    'node2vec_rw': None}

for embedding_name in nodes_embeddings:
    nodes_embeddings[embedding_name] = pickle.load(open('edges/'+embedding_name+'_all_embeddings.pickle', 'rb'))

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, dataset, dataset_split, nodes_embeddings):
        'Initialization'
        X, Y = [], []
        for doc in dataset[dataset_split]:
            
        
        self.X = X
        self.y2index = {l: i for i, l in enumerate(labels)}
        self.Y = Y
        self.labels = labels
        assert(len(X) == len(Y))

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.X)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        x = self.X[index].to_dense().clone().detach() #.to('cuda') # [:voc_size]
        y = self.y2index[self.Y[index]]
        
        return x, y